In [1]:
import re

In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
URL = 'https://www.lamudi.com.ph/buy/'
page = requests.get(URL)

In [3]:
page

<Response [403]>

In [95]:
from bs4 import BeautifulSoup
import requests


url = 'https://www.lamudi.com.ph/buy/?page=0'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
}

html_page = requests.get(url, headers=headers).text
soup = BeautifulSoup(html_page, "html.parser")

In [21]:
house_containers= soup.find_all('div', class_="row ListingCell-row")

In [22]:
[print(i,j) for i,j in enumerate(house_containers[5].find_all('a', class_="js-listing-link"))]

0 <a class="js-listing-link" data-position="6" data-sku="HO5E175309A4E99PH" href="https://www.lamudi.com.ph/modern-minimalist-house-for-sale-in-matina-davao-city.html" target="_blank" title="Modern Minimalist House for Sale in Matina Davao City">
<div class="ListingCell-Featured transparent">
<span class="ListingCell-Featured-text icon-featured">
                                    Featured
                                </span>
</div>
<div class="ListingCell-ImageCount">
<span class="ListingCell-ImageCount-text icon-camera">
                                23
                            </span>
</div>
<div data-listing-id="38861736" id="listingVideoIcon-38861736">
</div>
<div class="ListingCell-image ImageSection-loading">
<img alt="single-family-house For Sale in Matina Aplaya, Davao, Davao del Sur" data-src="https://static-ph.lamudi.com/static/media/bm9uZS9ub25l/2x2x2x380x244/f9729e13ee0904.jpg" src="/img/placeholder-image.svg"/>
</div>
</a>
1 <a class="js-listing-link" data-positi

[None, None, None, None, None, None, None, None]

In [9]:
# house_containers[0].find_all('a', class_="js-listing-link")[0]
#re.findall(r"(https\:.*)(?=\"\starget\=)",str(house_containers[0].find_all('a', class_="js-listing-link")[0]))[0]

In [10]:
#str(house_containers[0].find_all('a', class_="js-listing-link")[0])

In [102]:
[print(i,j) for i,j in enumerate(house_containers[5].find_all('span'))]

0 <span class="ListingCell-Featured-text icon-featured">
                                    Featured
                                </span>
1 <span class="ListingCell-ImageCount-text icon-camera">
                                9
                            </span>
2 <span class="trustedAgent icon-check">
                                                        Certified Partner
                                                    </span>
3 <span class="icon-pin"></span>
4 <span>
                                
                                MOA, Pasay
                            </span>
5 <span class="KeyInformation-value_v2">
                                        1
                                    </span>
6 <span class="KeyInformation-label_v2">Bedrooms</span>
7 <span class="KeyInformation-value_v2">
                                        1
                                    </span>
8 <span class="KeyInformation-label_v2">Baths</span>
9 <span class="KeyInformation-value_v2"

[None, None, None, None, None, None, None, None, None, None, None]

In [87]:
property_df = pd.DataFrame({"Address":None, "Price":None, "Bedrooms":None, "Baths":None, "Floor area":None, "Land Size":None})
#property_dict = {"Address":None, "Price":None, "Bedrooms":None, "Baths":None, "Floor area":None, "Land Size":None}

p_label_list= []
address= "Address"
price= "Price"
others= [i.text.strip() for i in house_containers[5].find_all('span', class_="KeyInformation-label_v2")]
p_label_list.append(address)
p_label_list.append(price)
p_label_list.extend(others)

p_values_list= []
address= " ".join(x for x in re.sub(r"[^a-zA-Z0-9,.]"," ",house_containers[8].find_all('span', class_="")[0].text).split())
price= house_containers[5].find_all('span', class_="PriceSection-FirstPrice")[0].text.strip()
others= [i.text.strip() for i in house_containers[5].find_all('span', class_="KeyInformation-value_v2")]
p_values_list.append(address)
p_values_list.append(price)
p_values_list.extend(others)


for key,value in zip(p_label_list,p_values_list):
    property_dict[key]=value

In [88]:
property_dict

{'Address': 'Philippines, Metro Manila, Makati, San Antonio, Malugay St San Antonio, Makati',
 'Price': '₱ 9,500,000',
 'Bedrooms': '4',
 'Baths': None,
 'Floor area': '200 m²',
 'Land Size': '326 m²'}

In [5]:
from time import sleep
from random import randint
%time

property_df = pd.DataFrame()
n_pages = 0


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
}
n_properties= 0
for page in range(0,10):
    n_pages += 1
    lamudi_url = 'https://www.lamudi.com.ph/rent/?page='+str(page)
    page_html = requests.get(lamudi_url, headers=headers).text
    soup = BeautifulSoup(page_html, 'html.parser')
    house_containers = soup.find_all('div', class_="row ListingCell-row")
    if house_containers != []:
        n_properties += len(house_containers)
        for container in house_containers:
            property_dict = {"ADDRESS":None, "PRICE":None, "BEDROOMS":None, "BATHS":None, "FLOOR AREA":None, "LAND SIZE":None}
            p_label_list= []
            address= "ADDRESS"
            price= "PRICE"
            others= [i.text.strip().upper() for i in container.find_all('span', class_="KeyInformation-label_v2")]
            p_label_list.append(address)
            p_label_list.append(price)
            p_label_list.extend(others)
            
            p_values_list= []
            try:
                address= " ".join(x for x in re.sub(r"[^a-zA-Z0-9,.]"," ",container.find_all('span', class_="")[0].text).split())
            except IndexError:
                address= "NA"
            try:
                price= container.find_all('span', class_="PriceSection-FirstPrice")[0].text.strip()
            except IndexError:
                price= "NA"
            others= [i.text.strip() for i in container.find_all('span', class_="KeyInformation-value_v2")]
            p_values_list.append(address)
            p_values_list.append(price)
            p_values_list.extend(others)

            for key,value in zip(p_label_list,p_values_list):
                property_dict[key]=value
                
            property_df = property_df.append(property_dict, ignore_index=True)
            
    else:
        break
    
    print(page)
 #   sleep(randint(1,2))
    
#print('You scraped {} pages containing {} properties.'.format(n_pages, n_properties))
#property_df.to_csv('property_details.csv', index = False)

Wall time: 0 ns
0
1
2
3
4
5
6
7
8
9


In [116]:
property_df.drop_duplicates().to_csv('property_details.csv', index = False)

In [6]:
property_df.head()

ADDRESS BATHS BEDROOMS  \
0                        D Macapagal Ave. MOA, Pasay  None     None   
1                Laguna Technopark Mampalasan, Bi an  None     None   
2    Philippines, Taguig City Fort Bonifacio, Taguig  None     None   
3                          Ocampo St. Malate, Manila     1        1   
4  Cavite Technopark, Barangay Sabang, Naic, Cavi...  None     None   

  FLOOR AREA LAND SIZE      PRICE FLOORS TOTAL ROOMS (TOTAL) AVAILABLE FROM  
0     173 m²      None         NA          NaN           NaN            NaN  
1    1200 m²      None  ₱ 300,000          NaN           NaN            NaN  
2    2000 m²      None   ₱ 20,000          NaN           NaN            NaN  
3      23 m²      None    ₱ 9,975          NaN           NaN            NaN  
4    1000 m²      None  ₱ 200,000          NaN           NaN            NaN

In [13]:
address= " ".join(x for x in re.sub(r"[^a-zA-Z0-9,.]"," ",house_containers[6].find_all('span', class_="")[0].text).split())
price= re.sub(r"[^a-zA-Z0-9,.]","",house_containers[6].find_all('span', class_="PriceSection-FirstPrice")[0].text)
area_sqm= [i for i in [re.findall(r"\d*\s*m\²",i) for i in [re.sub(r"[^a-zA-Z0-9²,.]","",i.text) for i in house_containers[11].find_all('span', class_="KeyInformation-value_v2")]] if len(i)!=0][0][0]
land_area= [i for i in [re.findall(r"\d*\s*m\²",i) for i in [re.sub(r"[^a-zA-Z0-9²,.]","",i.text) for i in house_containers[11].find_all('span', class_="KeyInformation-label_v2")]] if len(i)!=0][0][0]



In [18]:
from time import sleep
from random import randint
%time
li_address = []
li_price = []
li_area = []
#properties = pd.DataFrame({'Address':None, 'Price':None, "Area":None})
n_pages = 0

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
}
n_properties= 0
for page in range(0,3):
    n_pages += 1
    lamudi_url = 'https://www.lamudi.com.ph/buy/?page='+str(page)
    page_html = requests.get(lamudi_url, headers=headers).text
    soup = BeautifulSoup(page_html, 'html.parser')
    house_containers = soup.find_all('div', class_="row ListingCell-row")
    if house_containers != []:
        n_properties += len(house_containers)
        for container in house_containers:
            # Price
            try:
                price= re.sub(r"[^a-zA-Z0-9,.]","",container.find_all('span', class_="PriceSection-FirstPrice")[0].text)
            except IndexError:
                price = None
            
            # Address
            try:
                address= " ".join(x for x in re.sub(r"[^a-zA-Z0-9,.]"," ",container.find_all('span', class_="")[0].text).split())
            except IndexError:
                address= None

            # Area square metre
            try:
                area_sqm= [i for i in [re.findall(r"\d*\s*m\²",i) for i in [re.sub(r"[^a-zA-Z0-9²,.]","",i.text) for i in container.find_all('span', class_="KeyInformation-value_v2")]] if len(i)!=0][0][0]
            except IndexError:
                area_sqm= None
            li_address.append(address)
            li_price.append(price)
            li_area.append(area_sqm)
            
    else:
        break
    
    print(page)
    sleep(randint(1,2))
    
print('You scraped {} pages containing {} properties.'.format(n_pages, n_properties))

Wall time: 0 ns
0
1
2
You scraped 3 pages containing 80 properties.


In [19]:
pd.DataFrame({'Address':li_address, 'Price':li_price, "Area":li_area})

Address      Price   Area
0                         Loyola Heights, Quezon City  3,540,542   41m²
1   Pioneer St. Corner Sheridan St. Buayang Bato, ...  5,288,029   34m²
2   Quirino Highway, Brgy.Talipapa Novaliches, Que...  6,284,668   54m²
3                              Langkaan I, Dasmari as  4,300,000  118m²
4                                      Malate, Manila  2,000,963   19m²
..                                                ...        ...    ...
75  EDSA cor. Madison St. Barangka Ilaya, Mandaluy...  5,200,000   20m²
76  Philippines, Para aque, Santo Ni o, Ninoy Aqui...  7,400,000   25m²
77                 Suba Basbas Road Mactan, Lapu Lapu  2,800,000   31m²
78                 A. S. Fortuna St. Banilad, Mandaue  3,403,179   23m²
79            South Forbes Golf City Inchican, Silang  5,861,250  225m²

[80 rows x 3 columns]

In [ ]:
from bs4 import BeautifulSoup
import requests
import json


url = 'https://www.lamudi.com.ph/buy/?page=100'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
}

html_page = requests.get(url, headers=headers)
soup = BeautifulSoup(html_page.content, "html5lib")

In [ ]:
for i in soup.find_all('script'):
    if type(i.text) == str and i.text.strip().startswith('dataLayer = [Object.assign('):
        t = i.text.strip()
        t = t[t.find('{'):]
        t = t[:1+t.rfind('}')]
        t= t[:-37]
        data=json.loads(t)
        break

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(data["ecommerce"]["impressions"])

In [ ]:
%%time

n_pages = 0

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
}

for page in range(1,101):
    n_pages += 1
    lamudi_url = 'https://www.lamudi.com.ph/buy/?page='+str(page)
    r = get(sapo_url, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_="row ListingCell-row")
    if house_containers != []:
        for container in house_containers:
            
            # Price            
            price = container.find_all('span')[2].text
            if price == 'Contacte Anunciante':
                price = container.find_all('span')[3].text
                if price.find('/') != -1:
                    price = price[0:price.find('/')-1]
            if price.find('/') != -1:
                price = price[0:price.find('/')-1]
            
            price_ = [int(price[s]) for s in range(0,len(price)) if price[s].isdigit()]
            price = ''
            for x in price_:
                price = price+str(x)
            prices.append(int(price))

            # Zone
            location = container.find_all('p', class_="searchPropertyLocation")[0].text
            location = location[7:location.find(',')]
            zone.append(location)

            # Title
            name = container.find_all('span')[0].text
            titles.append(name)

            # Status
            status = container.find_all('p')[5].text
            condition.append(status)

            # Area
            m2 = container.find_all('p')[9].text
            if m2 != '-':
                m2 = m2.replace('\xa0','')
                m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                areas.append(m2)
                
            else:
                m2 = container.find_all('p')[7].text
                if m2 != '-':
                    m2 = m2.replace('\xa0','')
                    m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                    areas.append(m2)
                else:
                    areas.append(m2)

            # Creation date
            date = pd.to_datetime(container.find_all('div', class_="searchPropertyDate")[0].text[21:31])
            created.append(date)

            # Description
            desc = container.find_all('p', class_="searchPropertyDescription")[0].text[7:-6]
            descriptions.append(desc)

            # url
            link = 'https://casa.sapo.pt/' + container.find_all('a')[0].get('href')[1:-6]
            urls.append(link)

            # image
            img = str(container.find_all('img')[0])
            img = img[img.find('data-original_2x=')+18:img.find('id=')-2]
            thumbnails.append(img)
    else:
        break
    
    sleep(randint(1,2))
    
print('You scraped {} pages containing {} properties.'.format(n_pages, len(titles)))